In [1]:
import numpy as np
import pandas as pd
from ..lib.data_loading import loadHeadersCSV
from ..lib.plotting import *
from ..lib.helpers import *

# Build all other dataframes from this one
headers = loadHeadersCSV('data/header_analyses.csv')

ModuleNotFoundError: No module named 'lib'

# Overview

## Sample size

In [ ]:
# Monthly count for all samples
monthly_count = headers.md5.resample('M').count()
monthly_count = monthly_count.rename('All samples')
plotByMonth(monthly_count,
            'monthly_count',
            'Monthly Count ($\mu$={:.0f}, $\Sigma$={})'.format(monthly_count.mean(), monthly_count.sum()))

## Duplicates

In [ ]:
unique_count = headers.drop_duplicates(subset='md5').md5.resample('M').count().rename('Unique samples')
duplicated_count = headers[headers.duplicated(subset='md5')].md5.resample('M').count().rename('Duplicated samples')
print('Total number of unique analysis: {}'.format(unique_count.sum()))
print('Total number of duplicated analysis: {}'.format(duplicated_count.sum()))
dups_percent = pd.concat([unique_count, duplicated_count], axis=1)
dups_percent = dups_percent.divide(dups_percent.sum(axis=1), axis=0)
plotByMonthPercentage(dups_percent, 'dups_percent', 'Unique vs duplicated samples by percentage (MD5 unique)')

In [ ]:
# Sanity check
assert unique_count.sum() + duplicated_count.sum() == len(headers)

# Distribution of file types

In [ ]:
# Different file types scanned
file_types = headers.file_type.unique()
print('Total unique file types: {}\n'.format(len(file_types)))
# for t in sorted(file_types):
#     print(t)

The high number of file types can be easily explained by looking at the types list. Many of the types result to the same general type, but differ due to factors like versioning, size (for images) and encoding.

## Executables

In [ ]:
# Some ELF/Mach-O files are not caught by "executable" filter, added manually
exec_crit = headers.file_type.map(lambda x: 'executable' in x.lower() or x.startswith('ELF') or x.startswith('Mach-O'))
exec_types = sorted(headers.file_type[exec_crit].unique())
print('Total unique file types with "executable": {}\n'.format(len(exec_types)))
for t in exec_types:
    print(t)

### Windows Portable Executables (32bit)
These analysis *generally* contain both static and dynamic analysis

In [ ]:
pe32_types = list(filter(lambda x: x.startswith('PE32 '), exec_types))
print('Total unique PE32 (32bit) types: {}\n'.format(len(pe32_types)))
for t in pe32_types:
    print(t)

### Other architectures/platforms

In [ ]:
not_pe32_types = list(filter(lambda x: not x.startswith('PE32 '), exec_types))
print('Total unique not PE32 (32bit) types: {}\n'.format(len(not_pe32_types)))
for t in not_pe32_types:
    print(t)

## Text files

This includes source files, scripts, html, etc.

In [ ]:
text_keywords = ['text', 'foxpro']
text_crit = headers.file_type.map(mapKeywords(text_keywords))
# Exclude what was considered executable type
text_types = sorted(headers.file_type[~exec_crit & text_crit].unique())
print('Total unique file types with "text": {}\n'.format(len(text_types)))
for t in text_types:
    print(t)

## Media (images, audio, video)

In [ ]:
media_keywords = ['image', 'audio', 'video', 'mpeg', 'bitmap', 'icon resource']
media_crit = headers.file_type.map(mapKeywords(media_keywords))
media_types = sorted(headers.file_type[~exec_crit & ~text_crit & media_crit].unique())
print('Total unique file types with {}: {}\n'.format(media_keywords, len(media_types)))
for t in media_types:
    print(t)

## Archives

In [ ]:
arch_keywords = ['archive', 'compressed']
arch_crit = headers.file_type.map(mapKeywords(arch_keywords))
arch_types = sorted(headers.file_type[~exec_crit & ~text_crit & ~media_crit & arch_crit].unique())
print('Total unique archive types: {}\n'.format(len(arch_types)))
for t in arch_types:
    print(t)

## Documents

In [ ]:
doc_keywords = ['pdf', 'microsoft ', 'composite document file', 'fdf document', 'opendocument']
doc_crit = headers.file_type.map(mapKeywords(doc_keywords))
doc_types = sorted(headers.file_type[~exec_crit & ~text_crit & ~media_crit & ~arch_crit & doc_crit].unique())
print('Total unique document types: {}\n'.format(len(doc_types)))
for t in doc_types:
    print(t)

## Remaining types

In [ ]:
# Used to tune the previous sections for missing types
other_types = sorted(headers.file_type[~exec_crit & ~text_crit & ~media_crit & ~arch_crit & ~doc_crit].unique())
print('Total unique remaining types: {}\n'.format(len(other_types)))
for t in other_types:
    print(t)

In [ ]:
exec_count = headers[exec_crit].drop_duplicates(subset='md5').md5.resample('M').count().rename('Executables')
text_count = headers[text_crit].drop_duplicates(subset='md5').md5.resample('M').count().rename('Text files')
media_count = headers[media_crit].drop_duplicates(subset='md5').md5.resample('M').count().rename('Media')
arch_count = headers[arch_crit].drop_duplicates(subset='md5').md5.resample('M').count().rename('Archives')
doc_count = headers[doc_crit].drop_duplicates(subset='md5').md5.resample('M').count().rename('Documents')
other_count = headers[~exec_crit & ~text_crit & ~media_crit & ~arch_crit & ~doc_crit].drop_duplicates(subset='md5').md5.resample('M').count().rename('Other')

monthly_percent = pd.concat([exec_count, doc_count, arch_count, text_count, other_count, media_count], axis=1)
monthly_percent = monthly_percent.divide(monthly_percent.sum(axis=1), axis=0)
plotByMonthPercentage(monthly_percent, 'monthly_percent_unique', 'File types by percentage (only unique MD5)')

In [ ]:
# Sanity check
assert len(file_types) == len(exec_types) + len(text_types) + len(media_types) + len(doc_types) + len(arch_types) + len(other_types)

# Antivirus Classification

The amount of vendors varies throughout the time, hence the choice of normalizing by dividing the positive classifications by the total amount of vendors at the time of the analysis.

The following accounts only for unique samples, for duplicated samples only the last analysis is taken into account. Samples that were unknown in VirusTotal are given a value of -1.

In [ ]:
# Keeping this in a separate cell, since it takes 'some' time to exec
av_col = headers.drop_duplicates(subset='md5', keep='last').antivirus
av_col = av_col.apply(normalize)

In [ ]:
av_col_known = av_col[av_col != -1]
av_col_unknown = av_col[av_col == -1]
print('Number of known (in VirusTotal) samples:\t{}'.format(av_col_known.count()))
print('Number of unknown samples:\t\t\t{}'.format(av_col_unknown.count()))
print('Mean for known samples:\t\t\t\t{:.2f}'.format(av_col_known.mean()))
print('Standard deviation for known samples:\t\t{:.2f}'.format(av_col_known.std()))

The main problem now is defining the threshold that classifies a sample as malware.

The following values may need tweaking...

In [ ]:
NOTMAL_THR = 0.1
MAL_THR = 0.3
av_col_notmal = av_col_known[av_col_known < NOTMAL_THR]
av_col_mal = av_col_known[av_col_known > MAL_THR]
av_col_grey = av_col_known[(NOTMAL_THR <= av_col_known) & (av_col_known <= MAL_THR)]
print('Malware:\t{} ({:.2f})'.format(av_col_mal.count(), av_col_mal.count()/av_col_known.count()))
print('Not malware:\t{} ({:.2f})'.format(av_col_notmal.count(), av_col_notmal.count()/av_col_known.count()))
print('Grey area:\t{} ({:.2f})'.format(av_col_grey.count(), av_col_grey.count()/av_col_known.count()))

# Sanity check
assert len(av_col_known) == len(av_col_notmal) + len(av_col_mal) + len(av_col_grey)

av_col_notmal = av_col_notmal.resample('M').count().rename('Not malware [0, {})'.format(NOTMAL_THR))
av_col_mal = av_col_mal.resample('M').count().rename('Malware ({}, 1]'.format(MAL_THR))
av_col_grey = av_col_grey.resample('M').count().rename('Grey area [{}, {}]'.format(NOTMAL_THR, MAL_THR))

malware_percent = pd.concat([av_col_mal, av_col_grey, av_col_notmal], axis=1)
malware_percent = malware_percent.divide(malware_percent.sum(axis=1), axis=0)
plotByMonthPercentage(malware_percent, 'malware_percent', 'Samples classified as malware by percentage (only unique MD5)')

## Evolution in duplicated samples

How long until an unknown sample becomes known?

Do samples tend to increase or decrease in classification (i.e. false positive rate, false negative)?

How much time do vendors take to identify new malware (i.e. time from 0.0 to some threshold)?

In [ ]:
dups_class = headers.copy()
dups_class['index'] = dups_class.index
# Apply the normalization again, to facilitate filtering
dups_class['antivirus'] = dups_class['antivirus'].apply(normalize)
# Keep only duplicated samples that changed the antivirus classification
dups_class = dups_class.drop_duplicates(['md5', 'antivirus'])
# Keep only duplicated that changed in time, don't have access to the fulltimestamp, hence same day samples
# won't be helpful
# dups_class = dups_class.drop_duplicates(['md5', 'index'], keep='last')
# Keep only duplicates
dups_class = dups_class[dups_class.duplicated(subset='md5', keep=False)]

### From unknown to known

Check how much time it takes for a sample to go from unknown to known, this doesn't take into account if the sample is actually malware or not.

In [ ]:
# If the first submission was unknown (-1), calculate the time it took until
# sample became known.
def unknown_to_known(sample):
    if sample[0] == -1:
        for idx, val in enumerate(sample[1:]):
            if val != -1:
                # We take the average between the submission when the sample was known and the
                # submission before that, to give the vendors some slack
                # + 1 because we're enumerating from [1:]
                avg_days = (sample.index[idx + 1] - sample.index[idx]).days / 2
                return (sample.index[idx + 1] - sample.index[0]).days - avg_days
    else:
        return -1.0
        
dups_unk_to_k = dups_class.groupby('md5').agg({'antivirus': unknown_to_known}).antivirus.rename('days')
dups_unk_to_k = (dups_unk_to_k[dups_unk_to_k.map(lambda x: x != -1)]).astype(int)

print('Mean (in days) for a sample to become known:\t{:.2f}'.format(dups_unk_to_k.mean()))
print('Median for a sample to become known:\t\t{:.2f}'.format(dups_unk_to_k.median()))
print('Standard deviation (in days):\t\t\t{:.2f}'.format(dups_unk_to_k.std()))

One must take into account that the only available information is from malwr.com, which means samples may become known sooner, but one only knows after the sample has been resubmitted.

### Tendency in the classification

Check if the tendency for duplicated samples is to increase or decrease, in terms of classification.

Given that the classification results are only shown for vendors that have seen the sample, an upward tendency reflects a false negative tendency, while a downward tendency indicates a false positive tendency.

In [ ]:
# Calculates the tendency for a given sample, based on duplicated submissions
# Returns 1 for positive tendency (later submissions have higher classification)
# Returns -1 for negative tendency (later submissions have lower classification)
# Returns 0 for no tendency (classification between submissions doesn't go above a threshold)
# Returns -2 if there aren't enough submissions to check for a tendency
def tendency(sample):
    from functools import reduce
    # If the submissions didn't vary by more than this threshold, no tendency
    CHANGE_THR = 0.05
    values = []
    
    for idx, val in sample.iteritems():
        if val != -1:
            values.append(val)
    # Not enough values to check for tendency
    if len(values) < 2:
        return -2
    
    diff = [y - x for x, y in zip(values, values[1:])]
    # The mean variation is above the threshold
    if reduce(lambda x, y: abs(x) + abs(y), diff)/len(diff) > CHANGE_THR:
        pos = sum(x > 0 for x in diff)
        neg = sum(x < 0 for x in diff)
        return 1 if pos > neg else -1
    else:
        return 0
        

dups_tendency = dups_class.groupby('md5').agg({'antivirus': tendency}).antivirus.rename('tendency').astype(int)

In [ ]:
pos_tend = dups_tendency[dups_tendency == 1]
neg_tend = dups_tendency[dups_tendency == -1]

print('Number of submissions with positive tendency: {} ({:.2f})'.format(pos_tend.count(), pos_tend.count()/dups_tendency.count()))
print('Number of submissions with negative tendency: {} ({:.2f})'.format(neg_tend.count(), neg_tend.count()/dups_tendency.count()))

The positive tendency for over 50% of duplicated samples shows that vendors do prefer to classify

that the sample takes some time to propagate between vendors, while the negative tendency around 2% reflects a low false positive rate correction, meaning that vendors do not tend to unclassify a sample as malware.

### Average time to classify a malware

How much time does it take for a sample to go from "not malware" to "malware". Assuming the previous threshold.

In [ ]:
# Calculate the days it took for a sample go from below the threshold
# to above the malware threshold. Only includes samples that were known but
# not classified as malware
def zero_to_malware(sample):
    if 0 <= sample[0] < NOTMAL_THR:
        for idx, val in enumerate(sample[1:]):
            if val > 0.3:
                # + 1 because we're enumerating from [1:]
                avg_days = (sample.index[idx + 1] - sample.index[idx]).days / 2
                return (sample.index[idx + 1] - sample.index[0]).days - avg_days
        # If the sample never goes above the threshold
        return -1
    else:
        return -1
    
dups_zero_to_mal = dups_class.groupby('md5').agg({'antivirus': zero_to_malware}).antivirus.rename('days')
dups_zero_to_mal = dups_zero_to_mal[dups_zero_to_mal != -1]
# print(dups_zero_to_mal)

print('Mean (in days) to go from below the threshold to above:\t{:.2f}'.format(dups_zero_to_mal.mean()))
print('Median to go from below the threshold to above:\t\t{:.2f}'.format(dups_zero_to_mal.median()))
print('Standard deviation:\t\t\t\t\t{:.2f}'.format(dups_zero_to_mal.std()))

Like before, the obtained values are limited by the rebumissions to malwr.com, hence giving some slack between the submission above the threshold and the one before.

# Focusing on PE32 samples

## Sample size

In [ ]:
# Monthly count for PE32 samples
crit_pe32 = headers.file_type.map(lambda x: str(x).startswith('PE32 '))
pe32 = headers[crit_pe32]
pe_monthly_count = pe32.md5.resample('M').count()
pe_monthly_count = pe_monthly_count.rename('PE32 samples')
plotByMonth(pe_monthly_count,
            'pe_monthly_count',
            'PE Monthly Count ($\mu$={:.0f}, $\Sigma$={})'.format(pe_monthly_count.mean(), pe_monthly_count.sum()))

## Antivirus classification

In [ ]:
# Keeping this in a separate cell, since it takes 'some' time to exec
av_col = pe32.drop_duplicates(subset='md5', keep='last').antivirus
av_col = av_col.apply(normalize)

In [ ]:
av_col_known = av_col[av_col != -1]
av_col_unknown = av_col[av_col == -1]
print('Number of known (in VirusTotal) samples:\t{}'.format(av_col_known.count()))
print('Number of unknown samples:\t\t\t{}'.format(av_col_unknown.count()))
print('Mean for known samples:\t\t\t\t{:.2f}'.format(av_col_known.mean()))
print('Standard deviation for known samples:\t\t{:.2f}'.format(av_col_known.std()))

In [ ]:
av_col_notmal = av_col_known[av_col_known < NOTMAL_THR]
av_col_mal = av_col_known[av_col_known > MAL_THR]
av_col_grey = av_col_known[(NOTMAL_THR <= av_col_known) & (av_col_known <= MAL_THR)]
print('Malware:\t{} ({:.2f})'.format(av_col_mal.count(), av_col_mal.count()/av_col_known.count()))
print('Not malware:\t{} ({:.2f})'.format(av_col_notmal.count(), av_col_notmal.count()/av_col_known.count()))
print('Grey area:\t{} ({:.2f})'.format(av_col_grey.count(), av_col_grey.count()/av_col_known.count()))

# Sanity check
assert len(av_col_known) == len(av_col_notmal) + len(av_col_mal) + len(av_col_grey)

av_col_notmal = av_col_notmal.resample('M').count().rename('Not malware [0, {})'.format(NOTMAL_THR))
av_col_mal = av_col_mal.resample('M').count().rename('Malware ({}, 1]'.format(MAL_THR))
av_col_grey = av_col_grey.resample('M').count().rename('Grey area [{}, {}]'.format(NOTMAL_THR, MAL_THR))

malware_percent = pd.concat([av_col_mal, av_col_grey, av_col_notmal], axis=1)
malware_percent = malware_percent.divide(malware_percent.sum(axis=1), axis=0)
plotByMonthPercentage(malware_percent, 'malware_percent', 'Samples classified as malware by percentage (only unique MD5)')

## Evolution in duplicated samples

In [ ]:
dups_class = pe32.copy()
dups_class['index'] = dups_class.index
# Apply the normalization again, to facilitate filtering
dups_class['antivirus'] = dups_class['antivirus'].apply(normalize)
# Keep only duplicated samples that changed the antivirus classification
dups_class = dups_class.drop_duplicates(['md5', 'antivirus'])
# Keep only duplicated that changed in time, don't have access to the fulltimestamp, hence same day samples
# won't be helpful
dups_class = dups_class.drop_duplicates(['md5', 'index'], keep='last')
# Keep only duplicates
dups_class = dups_class[dups_class.duplicated(subset='md5', keep=False)]

### From unknown to known

In [ ]:
# If the first submission was unknown (-1), calculate the time it took until
# sample became known.
def unknown_to_known(sample):
    if sample[0] == -1:
        for idx, val in enumerate(sample[1:]):
            if val != -1:
                # We take the average between the submission when the sample was known and the
                # submission before that, to give the vendors some slack
                # + 1 because we're enumerating from [1:]
                avg_days = (sample.index[idx + 1] - sample.index[idx]).days / 2
                return (sample.index[idx + 1] - sample.index[0]).days - avg_days
    else:
        return -1.0
        
dups_unk_to_k = dups_class.groupby('md5').agg({'antivirus': unknown_to_known}).antivirus.rename('days')
dups_unk_to_k = (dups_unk_to_k[dups_unk_to_k.map(lambda x: x != -1)]).astype(int)

print('Mean (in days) for a sample to become known:\t{:.2f}'.format(dups_unk_to_k.mean()))
print('Standard deviation (in days):\t\t\t{:.2f}'.format(dups_unk_to_k.std()))

### Tendency in the classification

In [ ]:
# Calculates the tendency for a given sample, based on duplicated submissions
# Returns 1 for positive tendency (later submissions have higher classification)
# Returns -1 for negative tendency (later submissions have lower classification)
# Returns 0 for no tendency (classification between submissions doesn't go above a threshold)
# Returns -2 if there aren't enough submissions to check for a tendency
def tendency(sample):
    from functools import reduce
    # If the submissions didn't vary by more than this threshold, no tendency
    CHANGE_THR = 0.05
    values = []
    
    for idx, val in sample.iteritems():
        if val != -1:
            values.append(val)
    # Not enough values to check for tendency
    if len(values) < 2:
        return -2
    
    diff = [y - x for x, y in zip(values, values[1:])]
    # The mean variation is above the threshold
    if reduce(lambda x, y: abs(x) + abs(y), diff)/len(diff) > CHANGE_THR:
        pos = sum(x > 0 for x in diff)
        neg = sum(x < 0 for x in diff)
        return 1 if pos > neg else -1
    else:
        return 0
        

dups_tendency = dups_class.groupby('md5').agg({'antivirus': tendency}).antivirus.rename('tendency')

In [ ]:
pos_tend = dups_tendency[dups_tendency == 1]
neg_tend = dups_tendency[dups_tendency == -1]

print('Number of submissions with positive tendency: {} ({:.2f})'.format(pos_tend.count(), pos_tend.count()/dups_tendency.count()))
print('Number of submissions with negative tendency: {} ({:.2f})'.format(neg_tend.count(), neg_tend.count()/dups_tendency.count()))

### Average time to classify a malware

In [ ]:
# Calculate the days it took for a sample go from below the threshold
# to above the malware threshold. Only includes samples that were known but
# not classified as malware
def zero_to_malware(sample):
    if 0 <= sample[0] < NOTMAL_THR:
        for idx, val in enumerate(sample[1:]):
            if val > 0.5:
                # + 1 because we're enumerating from [1:]
                avg_days = (sample.index[idx + 1] - sample.index[idx]).days / 2
                return (sample.index[idx + 1] - sample.index[0]).days - avg_days
        # If the sample never goes above the threshold
        return -1
    else:
        return -1
    
dups_zero_to_mal = dups_class.groupby('md5').agg({'antivirus': zero_to_malware}).antivirus.rename('days').astype(int)
dups_zero_to_mal = dups_zero_to_mal[dups_zero_to_mal != -1]

print('Mean (in days) to go from below the threshold to above: {:.2f}'.format(dups_zero_to_mal.mean()))
print('Median to go from below the threshold to above: {:.2f}'.format(dups_zero_to_mal.median()))
print('Standard deviation (in days): {:.2f}'.format(dups_zero_to_mal.std()))